In [1]:
import transformers
import pandas as pd
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from datasets import Dataset
import re

2023-12-04 19:40:35.341948: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 19:40:35.384729: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 19:40:35.384771: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 19:40:35.384798: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 19:40:35.393362: I tensorflow/core/platform/cpu_feature_g

In [2]:
test_data = pd.read_csv('hpc_space/test_data_5_augmented.csv')

In [3]:
val_data = pd.read_csv('hpc_space/val_data_5.csv')

In [4]:
def clean_txt(text):
    text = re.sub("'", "", text)
    text = re.sub("(\\W)+", " ", text)
    return text

In [5]:
test_data['text'] = test_data.text.apply(clean_txt)

In [6]:
val_data['text'] = val_data.text.apply(clean_txt)

In [7]:
# def replace_gendered_words(text, gender):
#     gendered_word_mapping = {
#         0: {"zij": "hij", "Zij": "Hij", "ze": "hij", "Ze": "Hij", "haar": "hem", "Haar": "Hem", "mw": "dhr", "Mw": "Dhr", "vrouw": "man", "Vrouw": "Man", "patiente": "patient", "Patiente": "Patient", "mevrouw": "meneer", "Mevrouw": "Meneer", "mevr": "mr", "Mevr": "Mr", "meisje": "jongen", "Meisje": "Jongen", "dame": "heer", "Dame": "Heer"},
#         1: {"hij": "zij", "Hij": "Zij", "hem": "haar", "Hem": "Haar", "zijn": "haar", "Zijn":"Haar", "dhr": "mw", "Dhr": "Mw", "man": "vrouw", "Man": "Vrouw", "patient": "patiente", "Patient": "Patiente", "meneer": "mevrouw", "Meneer": "Mevrouw", "mr": "mevr", "Mr": "Mevr", "jongen": "meisje", "Jongen": "Meisje", "heer": "dame", "Heer": "Dame", "hr":"mw", "Hr": "Mw"},
#     }

#     words = text.split()

#     for i in range(len(words)):
#         word = words[i]
#         if word in gendered_word_mapping[gender]:
#             words[i] = gendered_word_mapping[gender][word]

#     return ' '.join(words)

# if __name__ == "__main__":

#     # Replace gendered words in the 'text' column based on the 'Geslacht' column
#     test_data['text'] = test_data.apply(lambda row: replace_gendered_words(row['text'], row['Geslacht']), axis=1)

#     # Print the modified DataFrame
#     print("Modified DataFrame:")
#     print(test_data)

In [8]:
# test_data['Geslacht'] = test_data['Geslacht'].replace({0: 1, 1: 0})

# # Print the modified DataFrame
# print("Modified DataFrame:")
# print(test_data)

In [9]:
# test_data['Combined'] = test_data['Combined'].replace({'0_1': '0_0', '0_0': '0_1', '1_0': '1_1', '1_1': '1_0'})

# # Print the modified DataFrame
# print("Modified DataFrame:")
# print(test_data)

In [10]:
# test_data_orig = pd.read_csv('hpc_space/MBERT1/test_data_mid_1.csv')

In [11]:
# test_data_orig['text'] = test_data_orig.text.apply(clean_txt)

In [12]:
# test_data = pd.concat([test_data_orig, test_data], ignore_index=True)

In [13]:
def get_split(text1):
    l_total = []
    chunk_size = 500
    overlap = 50

    words = text1.split()

    for start_idx in range(0, len(words), chunk_size - overlap):
        end_idx = start_idx + chunk_size
        l_parcial = words[start_idx:end_idx]
        l_total.append(" ".join(l_parcial))

    return l_total

In [14]:
test_data.loc[:, 'text_split'] = test_data['text'].apply(get_split)
test_data.head()

,text,outcome,Geslacht,Combined,text_split
0,pte en moeder gezien voor pre intake met PERS...,0,0,0_0,[pte en moeder gezien voor pre intake met PERS...
1,Overleg met PERSOON 1 psychiater HIC patiente...,0,0,0_0,[Overleg met PERSOON 1 psychiater HIC patiente...
2,Opnamegesprek dagdienst Aanwezigen patiente p...,0,0,0_0,[Opnamegesprek dagdienst Aanwezigen patiente p...
3,1 Psychische problemen Opener mimiek dan voorh...,0,1,0_1,[1 Psychische problemen Opener mimiek dan voor...
4,PERSOON 1 leek bij controle te slapen 1 Psych...,0,0,0_0,[PERSOON 1 leek bij controle te slapen 1 Psych...


In [15]:
val_data.loc[:, 'text_split'] = val_data['text'].apply(get_split)
val_data.head()

,text,outcome,Geslacht,Combined,text_split
0,Kennedy scores tbv teamoverleg 1 Psychische pr...,0,0,0_0,[Kennedy scores tbv teamoverleg 1 Psychische p...
1,1 Psychische problemen Mw kwam om ongeveer 14 ...,0,0,0_0,[1 Psychische problemen Mw kwam om ongeveer 14...
2,1 Psychische problemen Bekend met adhd pdd nos...,0,0,0_0,[1 Psychische problemen Bekend met adhd pdd no...
3,PERSOON 1 kinder en jeugdpsychiater Jeugd ACT...,0,1,0_1,[PERSOON 1 kinder en jeugdpsychiater Jeugd ACT...
4,Op verzoek van PERSOON 1 verwijzing 28 01 19 ...,0,0,0_0,[Op verzoek van PERSOON 1 verwijzing 28 01 19 ...


In [16]:
test_l = []
test_label_l = []
test_index_l = []
geslacht_l = []
for idx, row in test_data.iterrows():
    for l in row['text_split']:
        test_l.append(l)
        test_label_l.append(row['outcome'])
        test_index_l.append(idx)
        geslacht_l.append(row['Geslacht'])

In [17]:
val_l = []
val_label_l = []
val_index_l = []
val_geslacht_l = []
for idx, row in val_data.iterrows():
    for l in row['text_split']:
        val_l.append(l)
        val_label_l.append(row['outcome'])
        val_index_l.append(idx)
        val_geslacht_l.append(row['Geslacht'])

In [18]:
test_df = pd.DataFrame({'text':test_l, 'label':test_label_l, 'Geslacht':geslacht_l, 'original_index':test_index_l})
test_df.head()

,text,label,Geslacht,original_index
0,pte en moeder gezien voor pre intake met PERSO...,0,0,0
1,afentoe in het weekend Ketamine geen speed of ...,0,0,0
2,bijzijn moeder 4 ADL functies en beroepsmatig ...,0,0,0
3,uit zichzelf contact aan te gaan 3 Risico op a...,0,0,0
4,Overleg met PERSOON 1 psychiater HIC patiente ...,0,0,1


In [19]:
val_df = pd.DataFrame({'text':val_l, 'label':val_label_l, 'Geslacht':val_geslacht_l, 'original_index':val_index_l})
val_df.head()

,text,label,Geslacht,original_index
0,Kennedy scores tbv teamoverleg 1 Psychische pr...,0,0,0
1,conrete poging ondernomen Afgelopen dagen uiti...,0,0,0
2,gedrag Denken lijkt normofreen coherent inhoud...,0,0,0
3,1 Psychische problemen Mw kwam om ongeveer 14 ...,0,0,1
4,bekend Medicatie bij opname Quetiapine MGA 1d1...,0,0,1


In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
tokenizer = AutoTokenizer.from_pretrained("JoppeK/BERTje_augmented_5")
test_texts = test_df['text'].tolist()
encoded_test = tokenizer(test_texts, padding=True, truncation=True, return_tensors='pt')

In [22]:
model = TFAutoModelForSequenceClassification.from_pretrained("JoppeK/BERTje_augmented_5")
# Set batch size and sequence length

batch_size = 16
max_seq_length = 512

test_df_with_predictions = test_df.copy()

# Convert PyTorch tensors to TensorFlow tensors in batches
num_samples = len(test_texts)
for i in range(0, num_samples, batch_size):
    batch_input_ids = tf.convert_to_tensor(encoded_test['input_ids'][i:i+batch_size])
    batch_attention_mask = tf.convert_to_tensor(encoded_test['attention_mask'][i:i+batch_size])
    batch_token_type_ids = tf.convert_to_tensor(encoded_test['token_type_ids'][i:i+batch_size])

    # Get model predictions for the current batch

    logits = model(batch_input_ids, attention_mask=batch_attention_mask, token_type_ids=batch_token_type_ids).logits

    # Convert logits to probabilities and labels
    probs = tf.nn.softmax(logits, axis=1)
    predicted_labels = tf.argmax(probs, axis=1)

    # Update test_df_with_predictions with the current batch's predictions
    batch_indices = list(range(i, min(i + batch_size, num_samples)))
    for idx, label, prob in zip(batch_indices, predicted_labels.numpy(), np.max(probs.numpy(), axis=1)):
        test_df_with_predictions.at[idx, 'predicted_label'] = label
        test_df_with_predictions.at[idx, 'predicted_probability'] = prob

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2023-12-04 19:41:26.537580: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14791 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2023-12-04 19:41:26.918525: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at JoppeK/BERTje_augmented_5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [23]:
test_df_with_predictions.drop(['text'], axis=1, inplace=True)

In [24]:
test_df_with_predictions.rename(columns={'predicted_probability': 'predicted_probabilities'}, inplace=True)

In [25]:
print(test_df_with_predictions)

      label  Geslacht  original_index  predicted_label  \
0         0         0               0              0.0   
1         0         0               0              0.0   
2         0         0               0              0.0   
3         0         0               0              0.0   
4         0         0               1              0.0   
...     ...       ...             ...              ...   
9825      0         1            1477              0.0   
9826      0         1            1477              0.0   
9827      0         1            1477              0.0   
9828      0         1            1477              0.0   
9829      0         1            1477              0.0   

      predicted_probabilities  
0                    0.998639  
1                    0.999000  
2                    0.996818  
3                    0.999128  
4                    0.997658  
...                       ...  
9825                 0.998578  
9826                 0.999252  
9827           

In [26]:
test_df_with_predictions.to_csv('BERTje_augmented5_chunk_augmentedtest_predictions.csv')

In [27]:
tokenizer = AutoTokenizer.from_pretrained("JoppeK/BERTje_augmented_5")
val_texts = val_df['text'].tolist()
encoded_val = tokenizer(val_texts, padding=True, truncation=True, return_tensors='pt')

In [28]:
model = TFAutoModelForSequenceClassification.from_pretrained("JoppeK/BERTje_augmented_5")
# Set batch size and sequence length

batch_size = 16
max_seq_length = 512

val_df_with_predictions = val_df.copy()

# Convert PyTorch tensors to TensorFlow tensors in batches
num_samples = len(val_texts)
for i in range(0, num_samples, batch_size):
    batch_input_ids = tf.convert_to_tensor(encoded_val['input_ids'][i:i+batch_size])
    batch_attention_mask = tf.convert_to_tensor(encoded_val['attention_mask'][i:i+batch_size])
    batch_token_type_ids = tf.convert_to_tensor(encoded_val['token_type_ids'][i:i+batch_size])

    # Get model predictions for the current batch

    logits = model(batch_input_ids, attention_mask=batch_attention_mask, token_type_ids=batch_token_type_ids).logits

    # Convert logits to probabilities and labels
    probs = tf.nn.softmax(logits, axis=1)
    predicted_labels = tf.argmax(probs, axis=1)

    # Update val_df_with_predictions with the current batch's predictions
    batch_indices = list(range(i, min(i + batch_size, num_samples)))
    for idx, label, prob in zip(batch_indices, predicted_labels.numpy(), np.max(probs.numpy(), axis=1)):
        val_df_with_predictions.at[idx, 'predicted_label'] = label
        val_df_with_predictions.at[idx, 'predicted_probability'] = prob

Some layers from the model checkpoint at JoppeK/BERTje_augmented_5 were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at JoppeK/BERTje_augmented_5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [29]:
val_df_with_predictions.drop(['text'], axis=1, inplace=True)

In [30]:
val_df_with_predictions.rename(columns={'predicted_probability': 'predicted_probabilities'}, inplace=True)

In [31]:
print(val_df_with_predictions)

      label  Geslacht  original_index  predicted_label  \
0         0         0               0              0.0   
1         0         0               0              0.0   
2         0         0               0              0.0   
3         0         0               1              0.0   
4         0         0               1              0.0   
...     ...       ...             ...              ...   
5003      0         1             737              0.0   
5004      0         1             737              0.0   
5005      0         0             738              0.0   
5006      0         0             738              0.0   
5007      0         0             738              0.0   

      predicted_probabilities  
0                    0.993488  
1                    0.996100  
2                    0.998200  
3                    0.998124  
4                    0.998875  
...                       ...  
5003                 0.999305  
5004                 0.997405  
5005           

In [32]:
val_df_with_predictions.to_csv('BERTje_augmented5_chunk_val_predictions.csv')

In [ ]:
test_df_with_predictions['predicted_label'].value_counts()

In [ ]:
# Group the predicted labels based on 'original_index'
grouped_predictions = test_df_with_predictions.groupby('original_index')['predicted_label'].apply(list)

# Merge the grouped predictions back to the original DataFrame
merged_df = test_df_with_predictions.merge(grouped_predictions, left_on='original_index', right_index=True)

# Rename the new column
merged_df.rename(columns={'predicted_label_y': 'grouped_predicted_labels'}, inplace=True)

# Drop unnecessary columns
merged_df.drop(['text', 'predicted_label_x'], axis=1, inplace=True)

# Remove duplicates and keep only one row per unique original_index
final_df = merged_df.drop_duplicates(subset='original_index')

In [ ]:
final_df.to_csv('hpc_space/Results/temp/' + 'BERTje_genderneutral_predictions_1_temp.csv', index=False)

In [ ]:
final_df = pd.read_csv("hpc_space/Results/temp/BERTje_genderneutral_predictions_1_temp.csv")
# Convert string representations of lists to actual lists
final_df['grouped_predicted_labels'] = final_df['grouped_predicted_labels'].apply(eval)

# Function to determine final prediction
def calculate_final_prediction(grouped_labels):
    # Calculate the proportion of 1s in the grouped labels
    proportion_of_ones = grouped_labels.count(1) / len(grouped_labels)

    # Set the threshold for making the final prediction
    threshold = 0.25

    # Make the final prediction based on the proportion
    if proportion_of_ones > threshold:
        return 1
    else:
        return 0

# Apply the function to each row and create a new column for final predictions
final_df['final_prediction'] = final_df['grouped_predicted_labels'].apply(calculate_final_prediction)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

# Calculate accuracy for the positive class (label = 1)
accuracy_positive = accuracy_score(final_df['label'], final_df['final_prediction'])

# Calculate precision for the positive class (label = 1)
precision_positive = precision_score(final_df['label'], final_df['final_prediction'], pos_label=1)

# Calculate recall for the positive class (label = 1)
recall_positive = recall_score(final_df['label'], final_df['final_prediction'], pos_label=1)

# Calculate F1 score for the positive class (label = 1)
f1_positive = f1_score(final_df['label'], final_df['final_prediction'], pos_label=1)

# Calculate precision for the negative class (label = 0)
precision_negative = precision_score(final_df['label'], final_df['final_prediction'], pos_label=0)

# Calculate recall for the negative class (label = 0)
recall_negative = recall_score(final_df['label'], final_df['final_prediction'], pos_label=0)

# Calculate F1 score for the negative class (label = 0)
f1_negative = f1_score(final_df['label'], final_df['final_prediction'], pos_label=0)

# Calculate the AUC
auc = roc_auc_score(final_df['label'], final_df['final_prediction'])

# Calculate the AUC for males (Geslacht = 1)
auc_male = roc_auc_score(final_df[final_df['Geslacht'] == 1]['label'], final_df[final_df['Geslacht'] == 1]['final_prediction'])

# Calculate the AUC for females (Geslacht = 0)
auc_female = roc_auc_score(final_df[final_df['Geslacht'] == 0]['label'], final_df[final_df['Geslacht'] == 0]['final_prediction'])


# Print the calculated metrics separately for both classes
print(f"Accuracy (Overall): {accuracy_positive:.4f}")
print(f"Precision (Positive): {precision_positive:.4f}")
print(f"Recall (Positive): {recall_positive:.4f}")
print(f"F1 Score (Positive): {f1_positive:.4f}")
print(f"Precision (Negative): {precision_negative:.4f}")
print(f"Recall (Negative): {recall_negative:.4f}")
print(f"F1 Score (Negative): {f1_negative:.4f}")
print(f"AUC: {auc:.4f}")
print(f"AUC (Male): {auc_male:.4f}")
print(f"AUC (Female): {auc_female:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix

# Calculate the confusion matrix for the entire dataset
cm = confusion_matrix(final_df['label'], final_df['final_prediction'])

# Calculate TPR and FPR for male (Geslacht = 1)
male_indices = final_df['Geslacht'] == 1
cm_male = confusion_matrix(final_df[male_indices]['label'], final_df[male_indices]['final_prediction'])

tpr_male = cm_male[1, 1] / (cm_male[1, 0] + cm_male[1, 1])
fpr_male = cm_male[0, 1] / (cm_male[0, 0] + cm_male[0, 1])

# Calculate TPR and FPR for female (Geslacht = 0)
female_indices = final_df['Geslacht'] == 0
cm_female = confusion_matrix(final_df[female_indices]['label'], final_df[female_indices]['final_prediction'])

tpr_female = cm_female[1, 1] / (cm_female[1, 0] + cm_female[1, 1])
fpr_female = cm_female[0, 1] / (cm_female[0, 0] + cm_female[0, 1])

# Print the calculated metrics separately for both classes
print(f"TPR (Male): {tpr_male:.4f}")
print(f"TPR (Female): {tpr_female:.4f}")
print(f"FPR (Male): {fpr_male:.4f}")
print(f"FPR (Female): {fpr_female:.4f}")

In [ ]:
final_df['combined'] = final_df['Geslacht'].astype(str) + '_' + final_df['label'].astype(str) + '_' + final_df['final_prediction'].astype(str)

# Get the count of combinations
combination_counts = final_df['combined'].value_counts()

# Print the counts
print("Combined Counts:")
print(combination_counts)

In [ ]:
final_df.to_csv('hpc_space/Results/BERTje/' + 'BERTje_genderneutral_predictions_1.csv', index=False)

In [ ]:
# Split the DataFrame into first_half and second_half
first_half = final_df.iloc[:len(final_df) // 2]
second_half = final_df.iloc[len(final_df) // 2:]

first_half.reset_index(drop=True, inplace=True)
second_half.reset_index(drop=True, inplace=True)

# Compare the 'final_prediction' values in the two halves and calculate value counts
mismatch_counts = (first_half['final_prediction'] != second_half['final_prediction']).value_counts()

# Create a DataFrame to display the mismatch counts
mismatches = pd.DataFrame({'Mismatches': mismatch_counts})
print(mismatches)